<a href="https://colab.research.google.com/github/ndq3004/machine_learning/blob/master/MachineLearningCoBan/RecommendationSystem/Content_baseRcmdSys.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from __future__ import print_function
import numpy as np
import pandas as pd

link = "/content/drive/My Drive/Dataset/"
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv(link + 'ml-100k/u.user', sep='|', names=u_cols)
n_users = users.shape[0]
print('Number of users:', n_users)
#Reading ratings file:
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings_base = pd.read_csv(link + 'ml-100k/ua.base', sep='\t', names=r_cols)
ratings_test = pd.read_csv(link + 'ml-100k/ua.test', sep='\t', names=r_cols)
rate_train = ratings_base.as_matrix()
rate_test = ratings_test.as_matrix()
print('Number of traing rates:', rate_train.shape[0])
print('Number of test rates:', rate_test.shape[0])


Number of users: 943
Number of traing rates: 90570
Number of test rates: 9430


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  from ipykernel import kernelapp as app


In [8]:
i_cols = ['movie id', 'movie title' ,'release date','video release date', 'IMDb URL',
'unknown', 'Action', 'Adventure', 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 
'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']

items = pd.read_csv(link + 'ml-100k/u.item', sep='|', names=i_cols, encoding='latin-1')
n_items = items.shape[0]

X0 = items.as_matrix()
X_train_counts = X0[:, -19:]

print('number of items:', n_items)

number of items: 1682


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [6]:
print(rate_train[:5, :])

[[        1         1         5 874965758]
 [        1         2         3 876893171]
 [        1         3         4 878542960]
 [        1         4         3 876893119]
 [        1         5         3 889751712]]


In [0]:
from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer(smooth_idf=True, norm ='l2')
X = transformer.fit_transform(X_train_counts.tolist()).toarray()


In [0]:
def get_items_rated_by_user(rate_matrix, user_id):
  """
  return (item_ids, scores)
  """
  y = rate_matrix[:,0] # all users
  # item indices rated by user_id
  # we need to +1 to user_id since in the rate_matrix, id starts from 1
  # but id in python starts from 0
  ids = np.where(y == user_id +1)[0]
  item_ids = rate_matrix[ids, 1] - 1 # index starts from 0
  scores = rate_matrix[ids, 2]
  return (item_ids, scores)

In [11]:
from sklearn.linear_model import Ridge
from sklearn import linear_model
d = X.shape[1] # data dimension
W = np.zeros((d, n_users))
b = np.zeros(n_users)
for n in range(n_users):
  ids, scores = get_items_rated_by_user(rate_train, n)
  model = Ridge(alpha=0.01, fit_intercept = True)
  Xhat = X[ids, :]
  model.fit(Xhat, scores)
  W[:, n] = model.coef_
  b[n] = model.intercept_

Yhat = X.dot(W) + b

n = 10
np.set_printoptions(precision=2) # 2 digits after .
ids, scores = get_items_rated_by_user(rate_test, n)
print('Rated movies ids :', ids )
print('True ratings :', scores)
print('Predicted ratings:', Yhat[ids, n])


Rated movies ids : [ 37 109 110 226 424 557 722 724 731 739]
True ratings : [3 3 4 3 4 3 5 3 3 4]
Predicted ratings: [3.18 3.13 3.42 3.09 3.35 5.2  4.01 3.35 3.42 3.72]


In [12]:
def evaluate(Yhat, rates, W, b):
  se = cnt = 0
  for n in range(n_users):
    ids, scores_truth = get_items_rated_by_user( rates, n)
    scores_pred = Yhat[ids, n]
    e = scores_truth - scores_pred
    se += (e*e).sum(axis = 0)
    cnt += e.size
  return np.sqrt(se/cnt)
print('RMSE for training: %.2f' %evaluate(Yhat, rate_train, W, b))
print('RMSE for test : %.2f' %evaluate(Yhat, rate_test, W, b))


RMSE for training: 0.91
RMSE for test : 1.27


In [2]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
!ls "/content/drive/My Drive/Dataset/ml-100k"

allbut.pl  u1.base  u2.test  u4.base  u5.test  ub.base	u.genre  u.occupation
mku.sh	   u1.test  u3.base  u4.test  ua.base  ub.test	u.info	 u.user
README	   u2.base  u3.test  u5.base  ua.test  u.data	u.item
